In [4]:
# normal RAG completes a single documentgrab
# this breaks up a query into 
# can cut down on hallucination

#RAG keeps it accurate and up to date with a single retrieval and genereation
# now the retreival process is dynamic. Build intermediatee retreival chains
# KILT out perfroms. Trickyness of query drives number of RAG quesries and reduces halu
# recent RAG is becoming iterative to break down complex taks, But really slows down response
# sub queries and qub anwwere with rejection sampling
# Only the most likely to be correct chain is used in training. 

# rejections sampling. Model elarns to predict each step in the chain. So its trained to be good at iterative retrieval
# best of n sampling tries liots of chain in p[arrele and pickjs the nbest one
# fine tuned on llama one for QA and one for kilt
# KILT: A Benchmark for Knowledge Intensive Language Tasks  ( from facebook)


In [5]:
!uv add smolagents[toolkit]


Resolved 331 packages in 0.47ms
Audited 326 packages in 0.09ms


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [6]:
!uv add huggingface_hub

Resolved 331 packages in 0.41ms
Audited 326 packages in 0.08ms


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [7]:
from smolagents import CodeAgent, InferenceClientModel

model = InferenceClientModel()  

# Create an agent with no tools
agent = CodeAgent(tools=[], model=model)

# Run the agent with a task
result = agent.run("What is the sum of the first 5 prime numbers")
print(result)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ What is the sum of the first 5 prime numbers                                                                    │
│                                                                                                                 │
╰─ InferenceClientModel - Qwen/Qwen2.5-Coder-32B-Instruct ────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  def is_prime(n):                                                                                                 
      if n <= 1:                                                                                                   
          return False                                                                                             
      if n <= 3:                                                                                                   
          return True                                                                                              
      if n % 2 == 0 or n % 3 == 0:                                                                                 
          return False                                                                                             
      i = 5                                                                                                        
      while i * i <= n:                                                                                            
          if n % i == 0 or n % (i + 2) == 0:                                                                       
              return False                                                                                         
          i += 6                                                                                                   
      return True                                                                                                  
                                                                                                                   
  def first_n_primes(n):                                                                                           
      primes = []                                                                                                  
      num = 2                                                                                                      
      while len(primes) < n:                                                                                       
          if is_prime(num):                                                                                        
              primes.append(num)                                                                                   
          num += 1                                                                                                 
      return primes                                                                                                
                                                                                                                   
  first_5_primes = first_n_primes(5)                                                                               
  sum_of_first_5_primes = sum(first_5_primes)                                                                      
  print(sum_of_first_5_primes)                                                                                     
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
28

Out: None

[Step 1: Duration 18.71 seconds| Input tokens: 1,905 | Output tokens: 221]

...

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 4 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  final_answer(28)                                                                                                 
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer: 28

[Step 4: Duration 9.90 seconds| Input tokens: 10,515 | Output tokens: 730]

28


In [9]:

# from smolagents import CodeAgent, InferenceClientModel, DuckDuckGoSearchTool



# model = InferenceClientModel(model_id="CoRAG-Llama3.1-8B-MultihopQA")
# agent = CodeAgent(
#     tools=[DuckDuckGoSearchTool()],
#     model=model,
# )

# # Now the agent can search the web!
# result = agent.run("What does stack overflow say about printing pretty in python jupyter lab")
# print(result)

In [28]:

from smolagents import Tool


class CoverLetterGenerator(Tool):
    name = "cover_letter_generator"
    description = """
    This tool intakes some resume material and a job posting and generates a cover letter"""
    inputs = {
        "resume_material": {
            "type": "list",
            "description": "resume material in the form of .txt or .pdf",
        },
        "job_posting: {
            "type": "string",
            "description": "job posting of interest",
        }
    }
    output_type = "string"

    def forward(self, resume_material: list[str], job_posting: str) -> str:
        from huggingface_hub import list_models

        model = next(iter(list_models(filter=task, sort="downloads", direction=-1)))
        return model.id


cover_letter_generator_tool = CoverLetterGenerator()


# Inspirational Developer Helper: A RAG System

In [1]:

from IPython.core.formatters import BaseFormatter
from IPython.display import display, HTML

class MultilineStringFormatter(BaseFormatter):
    def __call__(self, obj):
        if isinstance(obj, str) and '\n' in obj:
            return f'<pre>{obj}</pre>'
        return None

# Register the custom formatter
ip = get_ipython()
ip.display_formatter.formatters['text/html'].for_type(str, MultilineStringFormatter())



In [2]:

import os
import json
from pprint import pprint

from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from huggingface_hub import InferenceClient


from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS


In [3]:

pdf_path = "../data/raw_documents/The Pragmatic Programmer_ your journey to mastery, 20th Anniversary Edition, 2nd Edition - The Pragmatic Programmer, 20th Anniversary Edition.pdf"
index_path = "../data/faiss_index"

loader = PyPDFLoader(pdf_path)
docs = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = text_splitter.split_documents(docs)


embeddings = HuggingFaceEmbeddings()
vectorstore = FAISS.from_documents(chunks, embeddings)

# Save the vector store
vectorstore.save_local(index_path)

# Load the vector store
new_vectorstore = FAISS.load_local(
       index_path, embeddings, allow_dangerous_deserialization=True
   )


llm_client = InferenceClient(
    provider="fireworks-ai",
)



/home/kerry/git/playground_project/playground_project/.venv/lib/python3.13/site-packages/torch/cuda/__init__.py:174: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 11040). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at /pytorch/c10/cuda/CUDAFunctions.cpp:109.)
  return torch._C._cuda_getDeviceCount() > 0


In [4]:

q = """What is the best choice for a developer when they feel stuck in their career. Give me tangible next steps and actions I can take today"""

retrieved_docs = new_vectorstore.similarity_search(query=q, k=5)
retrieved_lines_with_distances = [(d.page_content) for d in retrieved_docs]


context = "\n".join([r for r in retrieved_lines_with_distances])
PROMPT = """
Use the following pieces of information enclosed in <context> tags to provide an answer to the question enclosed in <question> tags.
<context>
{context}
</context>
<question>
{question}
</question>
"""
prompt = PROMPT.format(context=context, question=q)

completion = llm_client.chat.completions.create(
    model="meta-llama/Llama-3.1-8B-Instruct",
    messages=[
        {
            "role": "user",
            "content": prompt
        }
    ],
)

pprint(completion.choices[0].message.content)

('Based on the provided context, the best choice for a developer who feels '
 'stuck in their career is to take proactive steps to change their situation. '
 'Here are some tangible next steps and actions you can take today:\n'
 '\n'
 '1. **Ask if you can work remotely**: If you want to work remotely, ask your '
 "employer if it's possible. If they say no, start looking for companies that "
 'offer remote work options.\n'
 '2. **Invest in yourself**: Set aside time to study new technologies, '
 'programming languages, or skills that interest you. This will help you stay '
 'current and open up new career opportunities.\n'
 '3. **Experiment with different environments**: Try new tools, technologies, '
 'or platforms to gain experience and broaden your skill set. For example, if '
 "you've only worked with Windows, try Linux or a different IDE.\n"
 '4. **Stay current**: Read news and posts online about technologies different '
 'from your current project. This will help you stay up-to-da

# If this was a real POC: Next Steps

1. Evaluation

   
    a. Build an evaluation dataset for what answers are good enough

   
    b. Set up tracking on runtime and memory usage

   
    c. Evaluate other model and parameter choices to keep baseline performance and optimize on memory and time costs

2. In particular I would like to play around with CoRAG


Last TODOs:
- [ ] Basic Cleanup
    - [ ] fix inference client credentials
    - [ ] Create environment setup script
    - [X] Clean up repo structure
    - [ ] Clean up the warnings
    - [ ] Load to github with appropriate controls
- [ ] Understand:
    - [ ] How does FAISS work and alternatives
    - [ ] What model did you choose
    - [ ] What might indicate that your chunk size is too small or doesnt have enough overlap
    - [ ] How would tweaking the prompt template change things
    - [ ] What other text splitters could you have used
    - [ ] Know the hyperparameters for all functions
- [ ] Improvements
    - [ ] Add local LLM option with script for set up
    - [ ] Make an agent that either:
        - [ ] generates some code
        - [ ] gives you inpiration
        - [ ] tells you the weather
    
